# Quantitative Risk Management

Click <a href="https://colab.research.google.com/github/Lolillosky/QuantRiskManagement/blob/main/NOTEBOOKS/13_CDS_Calibration.ipynb">
    <img src="https://upload.wikimedia.org/wikipedia/commons/d/d0/Google_Colaboratory_SVG_Logo.svg" width="30" alt="Google Colab">
</a> to open this notebook in Google Colab.


In this exercise, you are going to bootstrap a zero coupon curve from par swap rates. With this curve and a given volatility, you are going to price a swaption using the Bachelier formula. Then, you are going to compute, with the help of AD, the sensitivities of the swaption with respect to zero coupon rates. Then, you will transform these to sensitivities to par swap rates.

## Import main libraries:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import pandas as pd


## Import the following libraries from the repository:

In [ ]:
import sys
#sys.path.append('../CODE')  # Adjust the path as necessary


from IPython.display import clear_output

!rm -r {'QuantRiskManagement'}

!git clone https://github.com/Lolillosky/QuantRiskManagement.git

import sys
sys.modules.pop
sys.path.insert(0,'QuantRiskManagement/CODE')

clear_output()


In [ ]:
import pytorch_spline
import pytorch_ir_curve


## Load the swap data

In '../DATA/IR_Swap.csv', you have par swap rates for different maturities:

In [ ]:
swap_rates = pd.read_csv('../DATA/IR_Swap.csv')
#swap_rates = pd.read_csv('/content/QuantRiskManagement/DATA/IR_Swap.csv')

swap_rates['Rate'] /= 100


In [ ]:
fitter = pytorch_ir_curve.CurveFitter(swap_rates['Mat (yrs)'].values, swap_rates['Rate'].values, 1.0)

zc_rates = fitter.fit()

ir_curve = pytorch_ir_curve.IR_Curve(torch.tensor(swap_rates['Mat (yrs)'].values), torch.tensor(zc_rates))

In [ ]:
plt.plot(swap_rates['Mat (yrs)'], swap_rates['Rate'], '.', label='Swap Rates')
plt.plot(swap_rates['Mat (yrs)'], zc_rates, '.', label='Zero Coupon Rates')

plt.legend()

plt.ylim((0))


In [ ]:
CDS_Data = pd.read_csv('../DATA/CDS_Data.csv', sep=';')

CDS_Data.head()

In [ ]:
curve_columns = ['Spread6M', 'Spread1Y', 'Spread2Y', 'Spread3Y', 'Spread4Y',
       'Spread5Y', 'Spread7Y', 'Spread10Y', 'Spread15Y', 'Spread20Y',
       'Spread30Y']

rec_column = ['Recovery']

In [ ]:
import re

def extract_years(input_string):
    match = re.search(r"(\d+)(M|Y)", input_string)
    if match:
        number = int(match.group(1))
        unit = match.group(2)
        if unit == 'Y':
            return number
        elif unit == 'M':
            return number / 12.0  # Convert months to fraction of a year
    return None


maturities = [extract_years(s) for s in curve_columns]
print(maturities)

In [ ]:
CDS_Data[curve_columns + rec_column] = CDS_Data[curve_columns + rec_column].replace('%', '',regex=True).astype(float) / 100

In [ ]:
#Telefonica S A	8FGCBA	SNRFOR	EUR	CR14


cds_curve = CDS_Data[(CDS_Data['ShortName'] == 'Telefonica S A') &
         (CDS_Data['Tier'] == 'SNRFOR') & 
         (CDS_Data['Ccy'] == 'EUR') & 
         (CDS_Data['DocClause'] == 'CR14')][curve_columns]

In [ ]:
plt.plot(maturities, cds_curve.values[0], 'o-')